# AI Driven Drone Navigation
This project aims to create a navigation system with the help of AI for DJI Tello drone using object detection and depth estimation for avoidance.

In [1]:
import arcade
import copy
import cv2
from djitellopy import Tello
import numpy as np
import threading
import time

from depth_dataset_tool import DepthDatasetTool
from map import Map
from my_models import YOLOv5, MiDaS, DepthDirectionDecision
from navigator import Navigator

e:\github\ai-driven-drone-navigation\venv\lib\site-packages\pyglet\image\codecs\wic.py:434: UserWarning: [WinError -2147417850] Cannot change thread mode after it is set
  warnings.warn(str(err))
e:\github\ai-driven-drone-navigation\venv\lib\site-packages\pyglet\media\codecs\wmf.py:838: UserWarning: [WinError -2147417850] Cannot change thread mode after it is set
  warnings.warn(str(err))


## Initialise variables

In [2]:
tello = Tello()
tello.connect()
tello.streamon()

[INFO] tello.py - 122 - Tello instance was initialized. Host: '192.168.10.1'. Port: '8889'.
[INFO] tello.py - 437 - Send command: 'command'
[ERROR] tello.py - 457 - 'utf-8' codec can't decode byte 0xcc in position 0: invalid continuation byte
[INFO] tello.py - 437 - Send command: 'command'
[INFO] tello.py - 461 - Response command: 'ok'
[INFO] tello.py - 437 - Send command: 'streamon'
[INFO] tello.py - 461 - Response streamon: 'ok'


In [3]:
# Minimum of more than 10% battery is required for DJI Tello to take off.
if tello.get_battery() <= 10:
    raise ValueError('DJI Tello battery is too low.')

In [4]:
depth_model = MiDaS()
depth_model.init()

Using cache found in C:\Users\Eris/.cache\torch\hub\intel-isl_MiDaS_master
Using cache found in C:\Users\Eris/.cache\torch\hub\intel-isl_MiDaS_master


In [5]:
object_model = YOLOv5()
object_model.init()

Using cache found in C:\Users\Eris/.cache\torch\hub\ultralytics_yolov5_master
YOLOv5  2022-4-4 torch 1.11.0+cu113 CUDA:0 (NVIDIA GeForce GTX 1080, 8192MiB)

Fusing layers... 
YOLOv5m6 summary: 378 layers, 35704908 parameters, 0 gradients
Adding AutoShape... 


In [6]:
ddd_model = DepthDirectionDecision()
ddd_model.load()

In [7]:
navigator = Navigator()

In [8]:
ddt = DepthDatasetTool()

In [9]:
env = Map(goal_pos=(500,550))
env.center_window()
env.setup()

In [10]:
# Global variables
frame_read = tello.get_frame_read()
in_flight = False
auto_navigate = False
depth = None
obj = None

## Initialise functions

In [11]:
def drone_cam():
    while 1:
        # Camera capture
        img = copy.copy(frame_read.frame)

        battery_status = tello.get_battery()
        cv2.putText(img, "Battery: {}%".format(battery_status), (5, 720 - 45), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
        temperature_status = tello.get_temperature()
        cv2.putText(img, "Temperature: {}C".format(temperature_status), (5, 720 - 5), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
        cv2.imshow('Tello Camera', img)

        # Keyboard control
        key = cv2.waitKey(1) & 0xff
        if key == 27:  # ESC
            break

In [12]:
def keyboard_control():
    global in_flight, auto_navigate
    # Input window
    img = cv2.imread('Images/keyboard.jpg')
    img = cv2.resize(img, (550,200))
    while 1:
        cv2.imshow('Keyboard Control', img)

        # Keyboard control
        key = cv2.waitKey(1) & 0xff
        if key == 27:  # ESC
            arcade.exit()
            break

        # Enable/disable auto navigation mode
        elif key == ord('q'):
            auto_navigate = not auto_navigate
        # Reset map
        elif key == ord('r'):
            env.setup()

        elif key == 32:  # Space
            if not in_flight:
                # Take-off drone
                tello.send_rc_control(0, 0, 0, 0)
                tello.takeoff()
                in_flight = True

            elif in_flight:
                # Land drone
                tello.send_rc_control(0, 0, 0, 0)
                try: # Handles dji tello unexpected landing
                    tello.land()
                except:
                    pass
                in_flight = False

        if not auto_navigate:
            if key == ord('w'):
                tello.move_forward(30)
            elif key == ord('s'):
                tello.move_back(20)
            elif key == ord('a'):
                tello.move_left(30)
            elif key == ord('d'):
                tello.move_right(30)
            elif key == ord('W'):
                tello.move_up(30)
            elif key == ord('S'):
                tello.move_down(30)
            elif key == ord('D'):
                tello.rotate_clockwise(30)
            elif key == ord('A'):
                tello.rotate_counter_clockwise(30)

In [13]:
def depth_estimation(show=True):
    global depth
    while 1:
        img = frame_read.frame
        # Depth estimation
        depth = depth_model.predict(img)
        if show:
            depth_norm = cv2.normalize(depth, None, 0, 1, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_64F)
            cv2.imshow('Tello Depth Camera', depth_norm)
            key = cv2.waitKey(1) & 0xff
            if key == 27:  # ESC
                break
        time.sleep(0.05)

In [14]:
def object_detection(show=True):
    global obj
    while 1:
        img = copy.copy(frame_read.frame)
        # Object detection
        obj = object_model.predict(img)
        if show:
            cv2.imshow('Tello Object Camera', img)
            key = cv2.waitKey(1) & 0xff
            if key == 27:  # ESC
                break
        time.sleep(0.05)

In [15]:
def auto_navigation():
    global in_flight, auto_navigate
    while 1:
        if in_flight and auto_navigate:
            lr, fb, ud, yaw = 0, 0, 0, 0
            env.set_speed(0.4)
            img = np.zeros((180, 1100, 1), dtype = "uint8")
            direction_str = None

            direction, near_obj_name, direction_value = navigator.get_direction(obj, ddd_model.predict(depth))
            if direction == 0:
                near_obj_name = None
                direction_str = 'forward'
                env.set_pos_direction(direction_str)
                fb = 20
            if direction == 1:
                direction_str = 'left'
                env.set_pos_direction(direction_str)
                lr = -20
            if direction == 2:
                direction_str = 'right'
                env.set_pos_direction(direction_str)
                lr = 20
            if direction == 3:
                direction_str = 'up'
                ud = 20
            if direction == 4:
                direction_str = 'down'
                ud = -20

            rot_direction = env.get_rot_direction()
            if rot_direction == 'left':
                yaw = -20 if env.get_target_angle_is_close() else -40
            if rot_direction == 'right':
                yaw = 20 if env.get_target_angle_is_close() else 40

            if env.get_red_light_status():
                lr, fb, ud = 0, 0, 0

            tello.send_rc_control(lr, fb, ud, yaw)

            if env.get_reach_goal():
                tello.send_rc_control(0, 0, 0, 0)
                tello.land()
                in_flight = False
                auto_navigate = False

            cv2.putText(img, 'Direction: {}'.format(direction_str), (25, 40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)
            cv2.putText(img, 'Rotation: {}'.format(rot_direction), (25, 80), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)
            cv2.putText(img, 'Avoiding: {}'.format(near_obj_name), (25, 120), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)
            cv2.putText(img, direction_value, (25, 160), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)
            cv2.imshow('Auto Navigation System', img)
            key = cv2.waitKey(1) & 0xff
            if key == 27:  # ESC
                break
        else:
            env.set_speed(0)

        angle = tello.get_yaw()
        if angle <= 0:
            open(".temp/rot", "w").write(str(-angle))
        else:
            open(".temp/rot", "w").write(str(360-angle))

In [16]:
def dataset_tool():
    new_depth_images = []
    new_depth_labels = []
    img = np.zeros((40, 300, 1), dtype="uint8")
    while 1:
        cv2.imshow('Dataset Collector', img)
        key = cv2.waitKey(1) & 0xff
        if key == 27:  # ESC
            break
        if key == ord('w'):
            new_depth_images.append(depth)
            new_depth_labels.append(0)
            print(len(new_depth_images))
        elif key == ord('s'):
            new_depth_images.append(depth)
            new_depth_labels.append(3)
            print(len(new_depth_images))
        elif key == ord('a'):
            new_depth_images.append(depth)
            new_depth_labels.append(1)
            print(len(new_depth_images))
        elif key == ord('d'):
            new_depth_images.append(depth)
            new_depth_labels.append(2)
            print(len(new_depth_images))
    ddt.collect(new_depth_images, new_depth_labels)

## Run threads

In [17]:
threading.Thread(target=drone_cam).start()
threading.Thread(target=keyboard_control).start()
threading.Thread(target=depth_estimation).start()
threading.Thread(target=object_detection).start()
threading.Thread(target=auto_navigation).start()
# threading.Thread(target=dataset_tool).start()

## Main

In [18]:
arcade.run()
cv2.destroyAllWindows()
if in_flight: tello.land()
tello.end()

Attempting to add texture: Images/finish-flag.png-0-0-0-0-False-False-False-Simple 
[2240175720000] No room for Images/finish-flag.png-0-0-0-0-False-False-False-Simple  size (501, 512)
[2240175720000] Resizing atlas from (512, 512) to (1024, 1024)
[2240175720000] Atlas resize took 0.013917499999998029 seconds
Attempting to add texture: Images/finish-flag.png-0-0-0-0-False-False-False-Simple 
Attempting to add texture: Images/drone.png-0-0-0-0-False-False-False-Simple 
[2240175720000] No room for Images/drone.png-0-0-0-0-False-False-False-Simple  size (720, 720)
[2240175720000] Resizing atlas from (1024, 1024) to (2048, 2048)
[2240175720000] Atlas resize took 0.008909799999997858 seconds
Attempting to add texture: Images/drone.png-0-0-0-0-False-False-False-Simple 
[INFO] tello.py - 470 - Send command (no response expected): 'rc 0 0 0 0'
Send command (no response expected): 'rc 0 0 0 0'
[INFO] tello.py - 437 - Send command: 'takeoff'
Send command: 'takeoff'
[INFO] tello.py - 461 - Respon